# 전처리된 데이터 EDA

In [ ]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import re
from konlpy.tag import Okt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf

train_file_path = "Data/processed_train2.csv"
test_file_path = "Data/processed_test2.csv"
submission_file_path = 'Data/submission.csv'
train_data = pd.read_csv(train_file_path)
print("Train Data Sample:")
print(train_data.head())

print("\nTrain Data Columns:")
print(train_data.columns)



test_data = pd.read_json(test_file_path).T
test_data.head()

submission_data = pd.read_csv(submission_file_path)


print("\nTest Data Sample:")
print(test_data.head())
print(test_data.columns)

# submission.csv 파일 읽기
submission_data = pd.read_csv(submission_file_path)
print("\nSubmission Data Sample:")
print(submission_data.head())
print(submission_data.columns)

In [ ]:
# train 데이터 구조 및 통계 정보 확인
print("\nTrain Data Info:")
print(train_data.info())

# test 데이터 구조 및 통계 정보 확인
print("\nTest Data Info:")
print(test_data.info())

# submission 데이터 구조 및 통계 정보 확인
print("\nSubmission Data Info:")
print(submission_data.info())

In [ ]:
# 결측치 확인
print("\nTrain Data Missing Values:")
print(train_data.isnull().sum())

print("\nTest Data Missing Values:")
print(test_data.isnull().sum())

print("\nSubmission Data Missing Values:")
print(submission_data.isnull().sum())

In [ ]:
# 결측치 확인
print("\nTrain Data Missing Values:")
print(train_data.isnull().sum())

print("\nTest Data Missing Values:")
print(test_data.isnull().sum())

print("\nSubmission Data Missing Values:")
print(submission_data.isnull().sum())

In [ ]:
label_col = 'class' if 'class' in train_data.columns else train_data.columns[-1]  # Assuming label column is the last one if unnamed
print("\nTrain Data Label Distribution:")
print(train_data[label_col].value_counts())

In [ ]:
# 텍스트 길이 분석
text_col = 'conversation' if 'conversation' in train_data.columns else train_data.columns[0]  # Assuming text column is the first one if unnamed

# 텍스트 데이터 타입 확인 및 정리
train_data['text_length'] = train_data[text_col].apply(lambda x: len(str(x)) if isinstance(x, str) else 0)
test_data['text_length'] = test_data['text'].apply(lambda x: len(str(x)) if isinstance(x, str) else 0)

print("\nTrain Data Text Length Stats:")
print(train_data['text_length'].describe())

print("\nTest Data Text Length Stats:")
print(test_data['text_length'].describe())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 텍스트 길이 분포 시각화
plt.figure(figsize=(12, 6))
sns.histplot(train_data['text_length'], bins=50, kde=True, color='blue', label='Train Data')
sns.histplot(test_data['text_length'], bins=50, kde=True, color='green', label='Test Data')
plt.legend()
plt.title('Text Length Distribution')
plt.xlabel('Text Length')
plt.ylabel('Frequency')
plt.show()

In [ ]:
!apt-get update      
!apt-get install fonts-nanum* #나눔글꼴 설치

In [ ]:
import matplotlib.font_manager as fm


fe = fm.FontEntry(fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', name='NanumGothic') #파일 저장되어있는 경로와 이름 설정
fm.fontManager.ttflist.insert(0, fe)  # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumGothic'}) #폰트설정

In [ ]:
from wordcloud import WordCloud
from konlpy.tag import Okt
import matplotlib.pyplot as plt
import pandas as pd

# 형태소 분석기 초기화
okt = Okt()

# 예시 폰트 경로
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'

# 텍스트 열 이름
text_col = 'conversation' if 'conversation' in train_data.columns else train_data.columns[0]

# 클래스별 단어 구름 시각화
for label in train_data['class'].unique():
    # 해당 클래스의 텍스트 추출
    text = " ".join(train_data[train_data['class'] == label][text_col].dropna().astype(str))
    
    # 형태소 분석을 통해 명사만 추출
    tokens = okt.nouns(text)
    
    # 명사 리스트를 문자열로 변환
    tokens_str = " ".join(tokens)
    
    # 단어 구름 생성
    wordcloud = WordCloud(font_path=font_path, width=800, height=400, background_color='white').generate(tokens_str)
    
    # 단어 구름 시각화
    plt.figure(figsize=(10, 5))
    plt.rcParams['font.family'] = 'NanumGothic'
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(f'Word Cloud for Label {label}')
    plt.axis('off')
    plt.show()



In [ ]:
# 클래스 분포 시각화
plt.figure(figsize=(10, 6))
sns.countplot(x='class', data=train_data)
plt.title('Class Distribution in Train Data')
plt.xlabel('Class')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# 텍스트 길이 계산
train_data['text_length'] = train_data['conversation'].apply(lambda x: len(x.split()))

# 클래스별 텍스트 길이 분포 시각화
plt.figure(figsize=(12, 6))
sns.boxplot(x='class', y='text_length', data=train_data)
plt.title('Text Length Distribution by Class')
plt.xlabel('Class')
plt.ylabel('Text Length')
plt.xticks(rotation=45)
plt.show()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# TF-IDF 벡터화
vectorizer = TfidfVectorizer(max_features=5000)
X_tfidf = vectorizer.fit_transform(train_data['conversation'])

# 클래스별 중요한 단어 시각화
classes = train_data['class'].unique()

for cls in classes:
    class_data = train_data[train_data['class'] == cls]['conversation']
    class_tfidf = vectorizer.transform(class_data)
    tfidf_means = np.mean(class_tfidf.toarray(), axis=0)
    tfidf_indices = np.argsort(tfidf_means)[::-1][:10]  # 상위 10개 단어 선택
    
    words = np.array(vectorizer.get_feature_names_out())[tfidf_indices]
    scores = tfidf_means[tfidf_indices]

    plt.figure(figsize=(10, 5))
    sns.barplot(x=scores, y=words)
    plt.title(f'Top 10 TF-IDF Words for Class: {cls}')
    plt.xlabel('TF-IDF Score')
    plt.ylabel('Word')
    plt.show()

In [ ]:
from collections import Counter
import seaborn as sns

# 형태소 분석기 초기화
okt = Okt()

# 모든 텍스트를 결합
all_text = " ".join(train_data['conversation'].dropna().astype(str))

# 형태소 분석을 통해 명사만 추출
nouns = okt.nouns(all_text)

# 명사 빈도수 계산
noun_counts = Counter(nouns)

# 상위 20개 단어 추출
top_nouns = noun_counts.most_common(20)

# 단어와 빈도 분리 (리스트로 변환)
words, counts = zip(*top_nouns)
words = list(words)
counts = list(counts)

# 단어 빈도 시각화
plt.figure(figsize=(12, 6))
sns.barplot(x=counts, y=words)
plt.title('Top 20 Words by Frequency')
plt.xlabel('Frequency')
plt.ylabel('Words')
plt.show()

In [ ]:
print("\nTrain Data Text Length Stats:")
print(train_data['text_length'].describe())

print("\nTest Data Text Length Stats:")
print(test_data['text_length'].describe())